In [1]:
from langchain_ollama import ChatOllama
from langchain_community.tools import tool
from langchain.agents import create_agent
from langchain_community.tools import StructuredTool
from pydantic import BaseModel, Field

In [2]:
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [6]:
class dataBMI(BaseModel):
    height : float = Field(description ="height of the person" )
    weight : float = Field(description ="weight of the person ")


def getBMI(height : float, weight : float)-> float:
     return weight/height*height


bmi_tool =  StructuredTool.from_function(
    func=getBMI,
    name = "bmi_tool",
    description="It calculates the bmi of the person given height and weight.",
    args_schema = dataBMI
)

In [ ]:
class bmiValue(BaseModel):
    bmi : float =Field(description="BMI value")

def isGood(bmi: float ) -> str:
    if bmi < 26 :
        return "Good"
    else:
        return "Bad"
    


isGood_tool =  StructuredTool.from_function(
    func=isGood,
    name = "isGood_tool",
    description="It decides wether the BMI of the person is good or bad",
    args_schema = dataBMI
)

In [13]:
sys_prompt = SystemMessage(content = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

In [18]:
model = ChatOllama(model="phi3:mini")

In [25]:
op_model = ChatOpenAI(model="gpt-4o-mini")

In [26]:
agent = create_agent(
    model=op_model,
    tools=[bmi_tool],
    system_prompt=sys_prompt
)

In [27]:
query = "What is the BMI of the person if his height is 187 and weight 85 ?"

result = agent.invoke(
    {"messages": [HumanMessage(content=query)]}
)

# The final response is the content of the last message
print(result["messages"][-1].content)

Thought: I now know the final answer
Final Answer: The BMI of the person with a height of 187 cm and weight of 85 kg is 24.3.


In [32]:
result

{'messages': [HumanMessage(content='What is the BMI of the person if his height is 187 and weight 85 ?', additional_kwargs={}, response_metadata={}, id='e89b7918-1c65-4aee-801b-383619cb91f4'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 211, 'total_tokens': 230, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CwhjqAydnsCyNqYRY26JbklIciCEq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bab66-b0d6-7e33-96b5-6d739a2a9392-0', tool_calls=[{'name': 'bmi_tool', 'args': {'height': 187, 'weight': 85}, 'id': 'call_b01Wa97xv5XgYfEgFr2evazN', 'type': 'tool_call'}], usage_metadata={'i

In [31]:
from pprint import pprint
print(sys_prompt)

content='\nAnswer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}\n' additional_kwargs={} response_metadata={}
